In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Multiplexores Cúanticos.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

In [ ]:
from qat.core.console import display
from qat.lang.AQASM import Program, H

In [ ]:
def GetResults(circuit):
    #Create a Job from the circuit
    job = circuit.to_job()
    #Import and create the linear algebra simulator
    from qat.qpus import LinAlg
    linalgqpu = LinAlg()
    #Submit the job to the simulator LinAlg and get the results
    result = linalgqpu.submit(job)
    QP = []
    States = []
    QA = []
    #Print the results
    for sample in result:
        #print("State %s probability %s amplitude %s" % (sample.state, sample.probability, sample.amplitude))
        QP.append(sample.probability)
        States.append(str(sample.state))
        QA.append(sample.amplitude)
    QP = pd.Series(QP, name='Probability')  
    States = pd.Series(States, name='States')  
    QA = pd.Series(QA, name='Amplitude') 
    pdf = pd.concat([States, QP, QA], axis=1)
    return pdf, circuit    
    

## 1. Rotaciones Controladas

Para realizar la carga de una función de probabilidad en un sistema cuántico la operación base es la rotación controlada por un estado previo. 

Vamos a hacer una prueba rápida 

In [ ]:
%load_ext qat.core.magic

In [ ]:
!pwd

In [ ]:
#Cargo mi paquete
sys.path.append('../../PhaseAmplification')

In [ ]:
from AuxiliarFunctions import get_histogram

In [ ]:
#Funcion de probabilidad que quiero cargar
def p(x):
    return x*x

In [ ]:
#Rotacion controlada de varios thetas 
def ScRs(Thetas):
    from qat.lang.AQASM import QRoutine
    qrout = QRoutine()
    #Numero de qbits para controlar las Thetas
    NumberOfQbits = int(np.log2(len(Thetas)))
    #Controlling qbits
    qcontrol = qrout.new_wires(NumberOfQbits)
    #Additional qbit where Rotation should be applied
    qtarget = qrout.new_wires(1)
    
    for j, theta in enumerate(Thetas):
        qprog.apply(crbs_gate(NumberOfQbits, j, theta), qcontrol+qtarget)
    return qrout

Lo primero que voy a hacer es generar un circuito cuántico de dos qbits. El primer qbit controlará la rotación controlada a aplicar sobre el segundo qbit. Como la rotación controlada sobre el segundo qbit dependerá del estado en el que esté el primero lo que haré será poner el primer qbit en una superposición equiprobable de los estados $|0\rangle$ y $|1\rangle$ usando una puerta Haddamard

In [ ]:
nqbits = 2

In [ ]:
qprog = Program()
qbits = qprog.qalloc(nqbits)
for i in range(nqbits-1):
    qprog.apply(H, qbits[i])


In [ ]:
circuit = qprog.to_circ()
%qatdisplay circuit --depth 2

Lo que quiero es aplicar una operación:

$$\hat{U}|q_0q_1\rangle= \{ \begin{array}{ll}
      \hat{R}_y(\theta_0)|q_1\rangle  & if \;|q_0\rangle = |0\rangle \\
      \hat{R}_y(\theta_1)|q_1\rangle  & if \;|q_0\rangle = |1\rangle \\
\end{array}$$

Es decir aplicar una rotacion controlada sobre $|q_1\rangle$ controlada por el **estado** $|q_0\rangle$.

Los angulos $\theta_i$ los calculoen la siguiente celda. Están sacados de la carga de probabilidades pero podrían ser dos ángulos aleatorios.


In [ ]:
print('Thetas: {}'.format(thetas))

In [ ]:
from dataloading_module import crbs_gate

In [ ]:
qprog = Program()
qbits = qprog.qalloc(nqbits)
for i in range(nqbits-1):
    qprog.apply(H, qbits[i])
qprog.apply(ScRs(2.0*thetas), qbits)

In [ ]:
circuitRC = qprog.to_circ()
%qatdisplay circuitRC --depth 0

In [ ]:
pdfRC, _ = GetResults(circuitRC)

In [ ]:
pdfRC

El problema fundamental de esta aproximación es el uso de Rotaciones Controladas que en general suelen ser operaciones difíciles de ejecutar en un ordenador cuántico ...

Además como para cada rotación preparo el estado que le corresponde meto muchas puertas X que posiblemente sean innecesarias ...

## 2.Quantum Multiplexors

Una forma de implementar rotaciones controladas por estados de forma mucho más eficiente es usando multiplexores cuanticos. En este caso se aplican rotaciones completas de ángulos sobre un qbit y se intercalan con operaciones **c-Not**. Esto genera circuitos menos aparatosos donde la complejidad la dan directamente las puertas **c-Not**. El problema base de los multiplexores es que las rotaciones no son directas hay que aplicar combinaciones inteligentes de los ángulos que se quieran rotar. 

Vamos a intentar usar el código de Juan para aplicar estas rotaciones

In [ ]:
nqbits = 2
a = 0.
b = 1.
nbins = 2**nqbits
centers, probs = get_histogram(p, a, b, nbins)
ListOfThetas= []
for m in range(nqbits):
    n_parts = 2**(m+1)
    edges = np.array([a+(b-a)*(i)/n_parts for i in range(n_parts+1)])
    p_zones = np.array([np.sum(probs[np.logical_and(centers>edges[i],centers<edges[i+1])]) for i in range(n_parts)])
    p_left = p_zones[[2*j for j in range(n_parts//2)]]
    p_tot = p_left + p_zones[[2*j+1 for j in range(n_parts//2)]]
    thetas = np.arccos(np.sqrt(p_left/p_tot))
    ListOfThetas.append(thetas)

In [ ]:
thetas

In [ ]:
#Nos traemos el multiplexor
from QuantumMultiplexors_Module import multiplexor_ry_m

In [ ]:
qprog = Program()
qbits = qprog.qalloc(nqbits)
for i in range(nqbits-1):
    qprog.apply(H, qbits[i])
multiplexor_ry_m(qprog, qbits, thetas, m, m)

In [ ]:
c = qprog.to_circ()

In [ ]:
%qatdisplay c

In [ ]:
from qat.lang.AQASM import QRoutine, RY, CNOT, build_gate
def multiplexor_ry_m_recurs(qprog, qbits, thetas, r_controls, i_target, sig=1.0):
    """
    Auxiliary function to create the recursive part of a multiplexor
    that applies an RY gate
    Parameters
    ----------

    qprog : Quantum QLM Program
        Quantum Program in which we want to apply the gates
    qbits : int
        Number of qubits of the quantum program
    thetas : np.ndarray
        numpy array containing the set of angles that we want to apply
    r_controls : int
        number of remaining controls
    i_target : int
        index of the target qubits
    sig : float
        accounts for wether our multiplexor is being decomposed with its
        lateral CNOT at the right or at the left, even if that CNOT is
        not present because it cancelled out
        (its values can only be +1. and -1.)
    """
    assert isinstance(r_controls, int), 'm must be an integer'
    assert isinstance(i_target, int), 'j must be an integer'
    assert sig == 1. or sig == -1., 'sig can only be -1. or 1.'
    if  r_controls > 1:
        # If there is more that one control, the multiplexor shall be
        # decomposed. It can be checked that the right way to
        # decompose it taking into account the simplifications is as

        #left angles
        x_l = 0.5*np.array(
            [thetas[i]+sig*thetas[i+len(thetas)//2] for i in range(len(thetas)//2)]
        )

        #right angles
        x_r = 0.5*np.array(
            [thetas[i]-sig*thetas[i+len(thetas)//2] for i in range(len(thetas)//2)]
        )
        multiplexor_ry_m_recurs(qprog, qbits, x_l, r_controls-1, i_target, 1.)
        qprog.apply(CNOT, qbits[i_target-r_controls], qbits[i_target])
        multiplexor_ry_m_recurs(qprog, qbits, x_r, r_controls-1, i_target, -1.)
        # Just for clarification, if we hadn't already simplify the
        # CNOTs, the code should have been
        # if sign == -1.:
        #   multiplexor_ry_m_recurs(qprog, qbits, x_l, r_controls-1, i_target, -1.)
        # qprog.apply(CNOT, qbits[i_target-r_controls], qbits[j])
        # multiplexor_ry_m_recurs(qprog, qbits, x_r, r_controls-1, i_target, -1.)
        # qprog.apply(CNOT, qbits[i_target-r_controls], qbits[i_target])
        # if sign == 1.:
        #   multiplexor_ry_m_recurs(qprog, qbits, x_l, r_controls-1, i_target, 1.)
    else:
        # If there is only one control just apply the Ry gates
        theta_positive = (thetas[0]+sig*thetas[1])/2.0
        theta_negative = (thetas[0]-sig*thetas[1])/2.0
        qprog.apply(RY(theta_positive), qbits[i_target])
        qprog.apply(CNOT, qbits[i_target-1], qbits[i_target])
        qprog.apply(RY(theta_negative), qbits[i_target])

def multiplexor_ry_m(qprog, qbits, thetas, r_controls, i_target):
    """
    Create a multiplexor that applies an RY gate on a qubit controlled
    by the former m qubits. It will have its lateral cnot on the right.
    Given a 2^n vector of thetas this function creates a controlled
    Y rotation of each theta. The rotation is controlled by the basis
    state of a 2^n quantum system.
    If we had a n qbit system and a
        - thetas = [thetas_0, thetas_1, ..., thetas_2^n-1]
    then the function applies
        - RY(thetas_0) controlled by state |0>_{n}
        - RY(thetas_1) controlled by state |1>_{n}
        - RY(thetas_2) controlled by state |2>_{n}
        - ...
        - RY(thetas_2^n-1) controlled by state |2^n-1>_{n}
    On the quantum system.
    Parameters
    ----------

    qprog : Quantum QLM Program
        Quantum Program in which we want to apply the gates
    qbits : int
        Number of qubits of the quantum program
    thetas : np.ndarray
        numpy array containing the set of angles that we want to apply
    r_controls: int
        number of remaining controls
    i_target: int
        index of the target qubits
    """
    multiplexor_ry_m_recurs(qprog, qbits, thetas, r_controls, i_target)
    qprog.apply(CNOT, qbits[i_target-r_controls], qbits[i_target])

In [ ]:
def staf(qbits, thetas):
    
    q_rout = QRoutine()
    reg = q_rout.new_wires(qbits)
    
    multiplexor_ry_m_recurs(q_rout, reg, thetas, 1, 1)
    return q_rout
    q_rout.apply(CNOT, reg[0], reg[len(thetas)])
    return q_rout
    
    
    

In [ ]:
q = staf(2, thetas)

In [ ]:
%qatdisplay q

In [ ]:
len(thetas)

In [ ]:
nqbits

In [ ]:
thetas

In [ ]:
thetas

In [ ]:
[thetas[i] for i in range(0, len(thetas),2)]

In [ ]:
[thetas[i] for i in range(1, len(thetas),2)]

In [ ]:
circuitQM = qprog.to_circ()
%qatdisplay circuitQM --depth 0

In [ ]:
pdfQM,_ = GetResults(circuitQM)

In [ ]:
pdfQM

In [ ]:
def Multiplexor(theta, controlbits):
    #El multiplexor base siempre son 2 qbits
    from qat.lang.AQASM import QRoutine, CNOT, RY
    qrout = QRoutine()
    #Cuantos bits los controlan:
    
    if m>1:
        qbits = qrout.new_wires(controlbits+1)
        thetas4Left = for i in range()
        thetas4Right = 
        pass
    else:
        #Un qbit de control
        qbits = qrout.new_wires(2)
        #Rotaciones. Solo hay dos
        thetaLeft = (theta[0]+theta[1])/2.0
        tethaRight = (theta[0]-theta[1])/2.0
        #Rotacion Izquierda
        qrout.apply(RY(thetaLeft), qbits[1])
        #Puerta c-NOT
        qrout.apply(CNOT, qbits[0], qbits[1])
        #Rotación Derecha
        qrout.apply(RY(tethaRight), qbits[1])
        #rout.apply(CNOT, qbits[0], qbits[1])
    return qrout

In [ ]:
qprog = Program()
qbits = qprog.qalloc(2)
for i in range(2-1):
    qprog.apply(H, qbits[i])
qprog.apply(Multiplexor(2.0*thetas, 1), qbits)

In [ ]:
circuitQMZ = qprog.to_circ()
%qatdisplay circuitQMZ --depth 1

In [ ]:
pdfQMZ,_ = GetResults(circuitQMZ)

In [ ]:
pdfQMZ

In [ ]:
    #Numero de qbits para controlar las Thetas
    NumberOfQbits = int(np.log2(len(Thetas)))
    #Controlling qbits
    qcontrol = qrout.new_wires(NumberOfQbits)
    #Additional qbit where Rotation should be applied
    qtarget = qrout.new_wires(1)
    
    for j, theta in enumerate(Thetas):
        qprog.apply(CRBS_gate(NumberOfQbits, j, theta), qcontrol+qtarget)
    return qrout

In [ ]:
sum(thetas)

In [ ]:
pdfQM,_ = GetResults(circuitQM)

In [ ]:
pdfQM